In [8]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [10]:
actions = [
    'talk',
    'Thank you',
    'Love you',
    'Nice to meet you',
    'Next time'
]

data = np.concatenate([
    np.load('dataset/seq_talk_1653410831.npy'),
    np.load('dataset/seq_Thank_you_1653410831.npy'),
    np.load('dataset/seq_love_you_1653410831.npy'),
    np.load('dataset/seq_Nice_to_meet_you_1653410831.npy'),
    np.load('dataset/seq_next_time_1653410831.npy')
], axis=0)

data.shape

(4010, 40, 100)

In [11]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(4010, 40, 99)
(4010,)


In [12]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(4010, 5)

In [13]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(3609, 40, 99) (3609, 5)
(401, 40, 99) (401, 5)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                41984     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total params: 44,229
Trainable params: 44,229
Non-trainable params: 0
_________________________________________________________________


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/200
113/113 [==============================] - 3s 17ms/step - loss: 114.7381 - acc: 0.2945 - val_loss: 33.9398 - val_acc: 0.4763

Epoch 00001: val_acc improved from -inf to 0.47631, saving model to models\model.h5
Epoch 2/200
113/113 [==============================] - 2s 17ms/step - loss: 132.4386 - acc: 0.3730 - val_loss: 37.3787 - val_acc: 0.4239

Epoch 00002: val_acc did not improve from 0.47631
Epoch 3/200
113/113 [==============================] - 2s 15ms/step - loss: 26.6007 - acc: 0.5198 - val_loss: 7.2128 - val_acc: 0.6733

Epoch 00003: val_acc improved from 0.47631 to 0.67332, saving model to models\model.h5
Epoch 4/200
113/113 [==============================] - 2s 15ms/step - loss: 25392.7930 - acc: 0.4165 - val_loss: 133882.7188 - val_acc: 0.2269

Epoch 00004: val_acc did not improve from 0.67332
Epoch 5/200
113/113 [==============================] - 2s 15ms/step - loss: 29921.2539 - acc: 0.2098 - val_loss: 8675.0576 - val_acc: 0.2020

Epoch 00005: val_acc did not im